## Raiders of the pottery GAN
### Usando Redes Generativas Adversarias para generación sintética de modelos 3D de materiales en arqueología

Los restos cerámicos proporcionan valiosa información que ayuda a datar el estrato arqueológico donde se encuentran, aportando <font color=violet>evidencias</font> de la producción local, relaciones comerciales, comportamiento de consumidores en poblaciones locales, etc. [1, 2]\cite{Orton93, Kampel2003}.

La <font color=violet>fragilidad de la cerámica</font> hace que la mayor parte del material recuperado en un yacimiento arqueológico no se corresponda con piezas completas por lo que la  <font color=violet>gran mayoría</font> del material disponible está  <font color=violet>fragmentado</font>.

<div style="text-align:center"><img src="https://media.giphy.com/media/uA147Pq6OvgCA/giphy.gif" /></div>

La colección está formada por <font color=violet>vasijas cerámicas de torno</font> encontradas en diversos yacimientos arqueológicos ibéricos ubicados en el Valle del Alto Gualquivir (España).

<div style="text-align:center"><img src="https://github.com/celiacintas/vasijas/blob/master/imagenes/dataset.png" width="400"/></div>


Está compuesta por unos miles de distintos recipientes, de los que se han registrado y clasificado <font color=violet>sus perfiles 2D proyectados</font> (vista lateral de cada una de ellas). La clasificación fue hecha en base a <font color=violet>criterios morfológicos</font> por un grupo de expertos~\cite{LucenaMTAP2014}.

## ¿Cómo generamos nuevas muestras 3D? 

<div style="text-align:center"><img src="https://github.com/celiacintas/vasijas/blob/master/imagenes/augmentation.png"/></div>



## ¿Qué bibliotecas de Python usamos para implementar nuestros experimentos?


<div style="text-align:center"><img src="https://media.giphy.com/media/JDmIrA84Pw1hK/giphy.gif" /></div>

In [ ]:
import ...

## Qué es entrenamiento adversario?

\begin{equation}
\min_{\text{roca}}\max_{\text{indiana}} V(\text{indiana}, \text{roca}) = \text{distancia entre ellos}
\end{equation}


<div style="text-align:center"><img src="https://media.giphy.com/media/UVqDh08BFyGfrx9tRU/giphy.gif" /></div>

<div style="text-align:center"><img src="https://github.com/NickleDave/generative-neural-nets-tutorial-scipy2020/blob/master/images/overview.png" /></div>

Imagen de \cite{DBLP:journals/corr/abs-1710-07035}


### Algunas notaciones básicas

- $x$ es un <font color=violet>elemento de nuestro dataset </font> (![](https://github.com/celiacintas/vasijas/blob/master/imagenes/amphora_1f3fa.png)).
- $D(x)$ es la salida de la red evaluadora (\includegraphics[scale=0.07]{images/detective.png}), que nos devuelve <font color=violet>la probabilidad</font> de que $x$ sea real (\includegraphics[scale=0.07]{images/amphora_1f3fa.png}) o generado (\includegraphics[scale=0.07]{images/robot-face_1f916.png})} . $D(x)$ debería ser un valor alto cuando $x$ proviene de los datos de entrenamiento y bajo cuando $x$ es salida del generador.
- $z$ es <font color=violet>el vector del espacio latente</font> muestreado sobre una distribución normal. $G(z)$ representa la función del generador (\includegraphics[scale=0.07]{images/robot-face_1f916.png}) que mapea el vector $z$ sobre el espacio de nuestros datos. 
- $D(G(z))$ es la probabilidad que la salida del generador sea un elemento real. $D$ trata de <font color=violet>maximizar la probabilidad</font> de clasificar correctamente elementos reales y falsos} ($\log D(x)$), y $G$ trata de <font color=violet> minimizar la probabilidad </font> de que $D$ clasifique sus elementos generados como falsos} $(\log (1 - D(G(x))$)
